# Bridge M3.2 → M3.3: Secure What You Deployed

**Duration:** 8-10 minutes  
**Type:** Within-Module Bridge  

---

## 🎯 Purpose

You just deployed your RAG application to Railway and Render with continuous deployment—but it's completely open to the internet. Before adding security in M3.3, this bridge validates that your deployment foundation is solid: URLs respond, auto-deploy works, domains are configured, and logs are visible. This checkpoint prevents building security on top of a broken deployment.

---

## 📚 Concepts Covered

- **Platform health validation:** Verify both Railway and Render deployments are live and accessible
- **Continuous deployment verification:** Confirm Git push triggers automatic deployment
- **SSL/domain readiness:** Check custom domain HTTPS configuration (optional)
- **Logging access:** Ensure platform logs are visible for debugging

---

## ✅ After Completing

You will have:
- ✓ Verified both deployment URLs return valid responses (or identified issues)
- ✓ Confirmed continuous deployment works from Git push to live deployment
- ✓ Documented custom domain status (done/pending/not configured)
- ✓ Accessed platform logs and recorded recent entries
- ✓ Understood the critical security gap and what M3.3 will address

---

## 🗺️ Context in Track

**Bridge:** L1.M3.2 (Cloud Deployment) → L1.M3.3 (API Security)  
**Previous:** M3.2 deployed to Railway/Render with CD enabled  
**Next:** M3.3 adds API key auth, rate limiting, and input validation  

---

### 💻 Run Locally (Windows)

```powershell
powershell -c "$env:PYTHONPATH='$PWD'; jupyter notebook"
```

---

## Section 1: RECAP - What You Just Accomplished (M3.2)

### ✅ Achievements Checklist

You completed M3.2: Cloud Deployment! Here's what you accomplished:

**✓ Deployed to two production platforms**  
- Railway and Render both running your RAG system with automatic HTTPS and custom domains

**✓ Implemented continuous deployment**  
- Every GitHub push automatically builds and deploys new versions without manual intervention

**✓ Experienced platform differences firsthand**  
- You understand Railway's speed versus Render's documentation, free tier cold starts versus paid tier always-on behavior

**✓ Built production-ready deployment pipeline**  
- You have a documented process and working infrastructure you can replicate for future projects

---

### 🎯 Portfolio Impact

This is **portfolio-worthy**. You have live URLs demonstrating production deployment skills. Employers want to see this.

---

## Section 2: VALIDATION CHECK 1 - Both URLs Respond

### ☐ Check: Both platforms deployed successfully

**Impact:** Confirms you can compare approaches and have redundancy  
**Validation:** Railway URL returns valid response, Render URL returns valid response  
**Fix if needed:** Review deployment logs for build failures, check environment variables

---

### Configuration

Enter your deployed URLs below:

**Configure your deployment URLs** and set offline mode if you're working without live deployments. Replace the placeholder URLs with your actual Railway and Render endpoints.

In [ ]:
# Configuration: Enter your deployed URLs
# Replace these with your actual deployment URLs from Railway and Render

RAILWAY_URL = "https://your-app.railway.app"  # Replace with your Railway URL
RENDER_URL = "https://your-app.onrender.com"  # Replace with your Render URL

# Optional: Set to True if testing offline (will use stub responses)
OFFLINE_MODE = False

print(f"✓ Railway URL configured: {RAILWAY_URL}")
print(f"✓ Render URL configured: {RENDER_URL}")
print(f"✓ Offline mode: {OFFLINE_MODE}")

**Test both deployment URLs** using curl to verify they respond with HTTP 2xx status codes. The function includes offline mode support and handles common failure scenarios like timeouts and server errors.

In [ ]:
import subprocess
import json
from datetime import datetime

def check_url_responds(url, platform_name):
    """
    Check if a URL responds with HTTP 200 or similar success status.
    Supports offline mode with stub responses.
    """
    print(f"\n{'='*60}")
    print(f"Testing {platform_name}: {url}")
    print(f"{'='*60}")
    
    if OFFLINE_MODE:
        print("⚠️  OFFLINE MODE - Using stub response")
        print("✓ Status: 200 (stub)")
        print("✓ Response: {'status': 'ok', 'message': 'Stub response for offline testing'}")
        return True
    
    # Skip check if using placeholder URLs
    if "your-app" in url:
        print("⚠️  SKIPPED - Placeholder URL detected")
        print("   Update RAILWAY_URL and RENDER_URL with your actual deployment URLs")
        return False
    
    try:
        # Use curl to test the URL
        result = subprocess.run(
            ['curl', '-s', '-o', '/dev/null', '-w', '%{http_code}', url],
            capture_output=True,
            text=True,
            timeout=30
        )
        
        status_code = result.stdout.strip()
        
        if status_code.startswith('2'):  # 2xx success
            print(f"✅ SUCCESS - Status: {status_code}")
            print(f"✓ {platform_name} is responding correctly")
            return True
        elif status_code.startswith('5'):  # 5xx server error
            print(f"❌ FAILED - Status: {status_code}")
            print(f"⚠️  Server error - Check deployment logs on {platform_name}")
            return False
        else:
            print(f"⚠️  WARNING - Status: {status_code}")
            print(f"   May need to check endpoint or authentication")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"❌ TIMEOUT - URL did not respond within 30 seconds")
        print(f"   May be cold start (common on free tier) - try again")
        return False
    except FileNotFoundError:
        print(f"⚠️  CURL NOT FOUND - Install curl or enable OFFLINE_MODE")
        return False
    except Exception as e:
        print(f"❌ ERROR - {str(e)}")
        return False

# Test both platforms
print(f"Starting URL validation checks at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nNote: Free tier services may take 30-60s to wake from cold start\n")

railway_ok = check_url_responds(RAILWAY_URL, "Railway")
render_ok = check_url_responds(RENDER_URL, "Render")

# Summary
print(f"\n{'='*60}")
print("VALIDATION SUMMARY - Both URLs Respond")
print(f"{'='*60}")
print(f"Railway: {'✅ PASS' if railway_ok else '❌ FAIL'}")
print(f"Render:  {'✅ PASS' if render_ok else '❌ FAIL'}")

if railway_ok and render_ok:
    print(f"\n🎉 CHECK PASSED - Both platforms are responding")
    print(f"✓ You have redundant deployments ready")
else:
    print(f"\n⚠️  CHECK INCOMPLETE - Review failed platforms")
    print(f"   1. Check deployment logs in platform dashboard")
    print(f"   2. Verify environment variables are set correctly")
    print(f"   3. Confirm build completed successfully")

---

## Section 3: VALIDATION CHECK 2 - Auto-Deploy from Git Push

### ☐ Check: Automatic deployments configured

**Impact:** Enables rapid iteration without manual deployment steps  
**Validation:** Push a small change to GitHub main branch, see automatic deployment within 5 minutes  
**Fix if needed:** Reconnect GitHub integration, check branch settings

---

### Test Instructions

To validate continuous deployment:

1. **Make a small change** to your repository (e.g., update README or add a comment)
2. **Commit and push** to the `main` branch (or configured deployment branch)
3. **Monitor platform dashboards** for automatic deployment trigger
4. **Wait 3-5 minutes** for build and deployment to complete
5. **Verify change is live** on your deployed URLs

### Record Your Results

**Record your auto-deploy test results** by setting the verification flags to True after you've confirmed that pushing to Git triggers automatic deployment on each platform.

In [ ]:
# Auto-Deploy Test Results
# Set these flags after testing continuous deployment

# Change these to True after you've verified auto-deployment works
RAILWAY_AUTO_DEPLOY_VERIFIED = False  # Set to True if Railway auto-deployed a change
RENDER_AUTO_DEPLOY_VERIFIED = False   # Set to True if Render auto-deployed a change

# Optional: Record details about your test
auto_deploy_notes = """
Test date: [Enter date]
Change made: [Describe the change you pushed]
Railway deploy time: [e.g., "3 minutes"]
Render deploy time: [e.g., "5 minutes"]
Issues encountered: [Note any problems or "None"]
"""

print(f"{'='*60}")
print("AUTO-DEPLOY VALIDATION RESULTS")
print(f"{'='*60}\n")

print(f"Railway auto-deploy: {'✅ VERIFIED' if RAILWAY_AUTO_DEPLOY_VERIFIED else '⏳ NOT TESTED'}")
print(f"Render auto-deploy:  {'✅ VERIFIED' if RENDER_AUTO_DEPLOY_VERIFIED else '⏳ NOT TESTED'}")

if RAILWAY_AUTO_DEPLOY_VERIFIED and RENDER_AUTO_DEPLOY_VERIFIED:
    print(f"\n🎉 CHECK PASSED - Both platforms have working CD")
    print(f"✓ You can push to Git and see changes automatically deployed")
elif not RAILWAY_AUTO_DEPLOY_VERIFIED and not RENDER_AUTO_DEPLOY_VERIFIED:
    print(f"\n⚠️  ACTION REQUIRED - Test auto-deployment")
    print(f"   Follow the instructions above to validate CD")
else:
    print(f"\n⚠️  PARTIAL - One platform verified, test the other")

print(f"\n{'='*60}")
print("Test Notes:")
print(f"{'='*60}")
print(auto_deploy_notes)

---

## Section 4: VALIDATION CHECK 3 - Custom Domain with SSL

### ☐ Check: Custom domain with SSL working (at least one platform)

**Impact:** Professional presentation for portfolio and demos  
**Validation:** Access your custom domain via HTTPS, certificate shows green lock  
**Fix if needed:** Review DNS propagation (can take 24-48 hours), verify domain configuration

---

### Instructions

1. If you configured a custom domain in M3.2, enter it below
2. If not, you can skip this section or mark it as "Pending" for future setup
3. Verify HTTPS works by visiting the domain in a browser

### Record Your Custom Domain Configuration

**Document your custom domain setup** by entering the domain name and setting the status. This check is optional but recommended for professional portfolio presentation.

In [ ]:
# Custom Domain Configuration
# Enter your custom domain(s) if you set them up

CUSTOM_DOMAIN = ""  # e.g., "myrag.example.com" or leave empty if not configured
DOMAIN_STATUS = "pending"  # Options: "done", "pending", "not_configured"

# Optional: Record SSL certificate details
domain_notes = """
Domain registrar: [e.g., GoDaddy, Namecheap, Cloudflare]
Platform using custom domain: [Railway or Render]
DNS configuration date: [Date you added DNS records]
SSL certificate issuer: [e.g., Let's Encrypt, shown in browser]
Any issues: [Note any problems or "None"]
"""

print(f"{'='*60}")
print("CUSTOM DOMAIN & SSL VALIDATION")
print(f"{'='*60}\n")

if CUSTOM_DOMAIN:
    print(f"Custom domain: {CUSTOM_DOMAIN}")
    print(f"Status: {DOMAIN_STATUS.upper()}")
    
    if DOMAIN_STATUS == "done":
        print(f"\n✅ CHECK PASSED - Custom domain with HTTPS configured")
        print(f"✓ Your deployment has a professional domain")
        print(f"✓ SSL certificate is active (verify green lock in browser)")
    elif DOMAIN_STATUS == "pending":
        print(f"\n⏳ IN PROGRESS - Domain configuration pending")
        print(f"   DNS propagation can take 24-48 hours")
        print(f"   Check platform dashboard for domain status")
        print(f"   Verify DNS records are correct (A, CNAME, etc.)")
    else:
        print(f"\n⚠️  Status unclear - Update DOMAIN_STATUS")
        
else:
    print(f"Status: NOT CONFIGURED")
    print(f"\n⚠️  OPTIONAL - Custom domain not required but recommended")
    print(f"   Platform URLs (*.railway.app, *.onrender.com) work fine")
    print(f"   Custom domains add professionalism for portfolios")
    print(f"   You can add this later if needed")

print(f"\n{'='*60}")
print("Configuration Notes:")
print(f"{'='*60}")
print(domain_notes)

---

## Section 5: VALIDATION CHECK 4 - Monitoring and Logging

### ☐ Check: Monitoring and logging set up

**Impact:** Visibility into deployment health and user activity  
**Validation:** View application logs in platform dashboard, see request history  
**Fix if needed:** Enable platform logging, configure log retention settings

---

### Instructions

1. Open your Railway and/or Render dashboard
2. Navigate to the Logs section for your deployment
3. Look for recent application logs (startup messages, requests, errors)
4. Copy the last 5-10 lines of logs into the cell below
5. If logs are not visible, check platform logging settings

### Record Log Samples

**Paste recent logs from your platform dashboards** to verify that logging is configured and accessible. Set the verification flags after confirming you can view application logs.

In [ ]:
# Platform Logs Validation
# Paste recent logs from your platform dashboards below

railway_logs_sample = """
[Paste last 5-10 lines from Railway logs here]

Example stub:
2025-11-06 10:23:45 | INFO: Starting application...
2025-11-06 10:23:46 | INFO: Uvicorn running on http://0.0.0.0:8000
2025-11-06 10:24:10 | INFO: GET /health 200 OK
"""

render_logs_sample = """
[Paste last 5-10 lines from Render logs here]

Example stub:
Nov 6 10:25:32 AM  INFO Starting service
Nov 6 10:25:33 AM  INFO Application ready
Nov 6 10:25:45 AM  INFO 200 GET /api/query
"""

# Set to True after you've verified logs are visible
RAILWAY_LOGS_VERIFIED = False
RENDER_LOGS_VERIFIED = False

print(f"{'='*60}")
print("PLATFORM LOGGING VALIDATION")
print(f"{'='*60}\n")

print("Railway Logs:")
print("-" * 60)
if RAILWAY_LOGS_VERIFIED:
    print("✅ Logs visible and accessible")
    print(railway_logs_sample[:200] + "..." if len(railway_logs_sample) > 200 else railway_logs_sample)
else:
    print("⏳ NOT VERIFIED - Check Railway dashboard")
    print("   Navigate to: Project → Deployments → Logs")
    
print("\n" + "="*60)
print("Render Logs:")
print("-" * 60)
if RENDER_LOGS_VERIFIED:
    print("✅ Logs visible and accessible")
    print(render_logs_sample[:200] + "..." if len(render_logs_sample) > 200 else render_logs_sample)
else:
    print("⏳ NOT VERIFIED - Check Render dashboard")
    print("   Navigate to: Service → Logs tab")

print("\n" + "="*60)
print("VALIDATION SUMMARY")
print("="*60)

if RAILWAY_LOGS_VERIFIED and RENDER_LOGS_VERIFIED:
    print("🎉 CHECK PASSED - Logging configured on both platforms")
    print("✓ You can monitor application health and debug issues")
elif RAILWAY_LOGS_VERIFIED or RENDER_LOGS_VERIFIED:
    print("⚠️  PARTIAL - One platform verified, check the other")
else:
    print("⏳ ACTION REQUIRED - Verify logs on both platforms")
    print("   Logs are essential for debugging production issues")

---

## Section 6: CALL-FORWARD - What's Next in M3.3

### 🚨 The Critical Security Gap

Your API is **completely open** to the internet. Anyone who discovers your URL can use your RAG system - for free, using your OpenAI credits.

### Cost & Risk Exposure

**Cost exposure:** Your OpenAI API costs roughly ₹0.30 per query with embeddings and generation.
- 1,000 automated queries = ₹300 in unwanted charges
- 10,000 bot queries = ₹3,000 before you notice

**Time risk:** Once compromised, you need to immediately revoke API keys and redeploy (2-4 hours of emergency response time).

**Reputation damage:** If employers try your demo URL and find it rate-limited or shut down due to abuse, you look unprofessional.

---

### 🔒 M3.3: API Development & Security

**Next module question:** *"How do you secure a public API without sacrificing performance?"*

You'll implement three critical security layers:

---

### 1️⃣ API Key Authentication

**What you'll build:**
- Generate cryptographically secure keys
- Store them hashed (never plaintext)
- Require authentication on all endpoints

**Complexity note:** 
- Implementation: 6-10 hours initially
- Requires ongoing key rotation and monitoring

---

### 2️⃣ Rate Limiting (Three-Tier)

**What you'll build:**
- Minute-level limits: Prevent burst attacks
- Hourly limits: Control sustained usage
- Burst limits: Allow reasonable traffic spikes
- Redis-based distributed tracking

**Trade-offs:**
- Adds 10-50ms per request for limit checking
- Requires Redis setup (adds $20-50/month for production)

---

### 3️⃣ Input Validation & Security Headers

**What you'll build:**
- Block injection attacks (SQL, XSS, prompt injection)
- Regex patterns for input sanitization
- Security headers for defense-in-depth (CORS, CSP, etc.)
- Logging to detect and respond to attacks

**Benefits:**
- Protects against most common attack vectors
- Industry-standard security patterns
- Production-ready hardened API

---

### 📊 Technical Preview

You'll use:
- **FastAPI's dependency injection** for authentication
- **Redis** for distributed rate limiting
- **Pydantic validators** for input sanitization
- **Security logging** to detect attacks

**Estimated time:** 37 minutes video + 90 minutes hands-on practice

---

### ✅ Readiness Check Complete

If you've validated all four deployment checks above, you're ready for M3.3!

**Your deployment is live but unprotected.** The next module will secure it properly.

---

**See you in M3.3: API Development & Security!** 🔒